In [2]:
from experiment_project.utils.initial.util import init_sys_env
from experiment_project.utils.files.read import read_yaml
import dspy
# init_sys_env()
secret_env_file = 'env_secret_config.yaml'

api_configs = read_yaml(secret_env_file)
model_config = api_configs.get('openai')
turbo = dspy.OpenAI(model=model_config.get('model'), max_tokens=2048,api_key=model_config.get('api_key'))
dspy.settings.configure(lm=turbo)

In [3]:
# program of thought example
import dspy

# 定义 Signature: 输入是两个字符串表示的数字，输出是它们的和
class AddTwoNumbers(dspy.Signature):
    a = dspy.InputField(desc="第一个数字", type=str)
    b = dspy.InputField(desc="第二个数字", type=str)
    sum = dspy.OutputField(desc="两个数字的和", type=str)

# 创建 PoT 对象
pot = dspy.ProgramOfThought(AddTwoNumbers)

# 定义问题，将数字转换为字符串
problem = {"a": str(10), "b": str(20)}  

# 解决问题
response = pot(**problem)

# 打印结果
print(response.sum)  # 输出: 30


30


In [4]:
turbo.inspect_history(n=3)




You will be given `a`, `b` and you will respond with `sum`.
Generating executable Python code that programmatically computes the correct `sum`.
After you're done with the computation, make sure the last line in your code evaluates to the correct value for `sum`.

---

Follow the following format.

A: 第一个数字

B: 第二个数字

Reasoning: Let's think step by step in order to ${produce the generated_code}. We ...

Code: python code that answers the question

---

A: 10

B: 20

Reasoning: Let's think step by step in order to Reasoning: Let's think step by step in order to produce the generated_code. We need to add the two given numbers `a` and `b` to find their sum. 

Code:
```python
a = 10
b = 20
sum = a + b
sum
```

The last line evaluates to the correct value for `sum`, which is 30.





Given the final code `a`, `b`, `final_generated_code`, `code_output`, provide the final `sum`.

---

Follow the following format.

A: 第一个数字

B: 第二个数字

Code: python code that answers the question

Code Output:

"\n\n\nYou will be given `a`, `b` and you will respond with `sum`.\nGenerating executable Python code that programmatically computes the correct `sum`.\nAfter you're done with the computation, make sure the last line in your code evaluates to the correct value for `sum`.\n\n---\n\nFollow the following format.\n\nA: 第一个数字\n\nB: 第二个数字\n\nReasoning: Let's think step by step in order to ${produce the generated_code}. We ...\n\nCode: python code that answers the question\n\n---\n\nA: 10\n\nB: 20\n\nReasoning: Let's think step by step in order to\x1b Reasoning: Let's think step by step in order to produce the generated_code. We need to add the two given numbers `a` and `b` to find their sum. \n\nCode:\n```python\na = 10\nb = 20\nsum = a + b\nsum\n```\n\nThe last line evaluates to the correct value for `sum`, which is 30.\x1b\n\n\n\n\n\nGiven the final code `a`, `b`, `final_generated_code`, `code_output`, provide the final `sum`.\n\n---\n\nFollow the following format.\n\nA: 第一个数字\n\nB: 第二个数字\

In [5]:
# 一个搜索模块
import dspy

# 1. 选择 "搜索引擎":  这里使用 ColBERTv2 模型

colbert = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts') 
dspy.settings.configure(rm=colbert)  # 配置 DSPy 使用这个模型

# 2. 创建 Retrieve 对象,  指定返回 3 个结果
retriever = dspy.Retrieve(k=4) 

# 3.  输入问题，获取答案
question = "openai是什么公司？"
results = retriever(question).passages  # 获取相关段落

# 打印结果
for i, passage in enumerate(results):
    print(f"{i+1}. {passage}") 


1. OpenAI | OpenAI is a non-profit artificial intelligence (AI) research company that aims to promote and develop friendly AI in such a way as to benefit humanity as a whole. The organization aims to "freely collaborate" with other institutions and researchers by making its patents and research open to the public. The founders (notably Elon Musk and Sam Altman) are motivated in part by concerns about existential risk from artificial general intelligence.
2. Kamaitachi opening | In shogi, the Kamaitachi opening (かまいたち戦法 "kamaitachi senpou") or Eishun opening (英春流 "eishun-ryuu") is an uncommon flexible opening.
3. Open Archives Initiative | The Open Archives Initiative (OAI) is an organization to develop and apply technical interoperability standards for archives to share catalog information (metadata). It attempts to build a "low-barrier interoperability framework" for archives (institutional repositories) containing digital content (digital libraries). It allows people (service provide

In [ ]:

# 定义一个用于基本问答的简单签名
class BasicQA(dspy.Signature):
    question = dspy.InputField()
    answer = dspy.OutputField(desc="经常在1到5个单词之间")

# Pass signature to ReAct module
react_module = dspy.ReAct(BasicQA,)

# Call the ReAct module on a particular input
question = '天空的颜色是什么？使用中文回答!'
result = react_module(question=question)

print(f"Question: {question}")
print(f"最终预测答案（经过ReAct过程后）: {result.answer}")

In [ ]:
turbo.inspect_history(n=3)


In [3]:
# ChainOfThought example
# 创建任务模块  
class MathQuestion(dspy.Module):  
    def __init__(self):  
        super().__init__()  
        # 使用 ChainOfThought 进行逐步推理  
        self.think = dspy.ChainOfThought('question -> answer')  
    def forward(self, question:str):    
        # 组合完整解决方案  
        return self.think(question= question)   
  
# 创建 MathSolver 实例  
math_question = MathQuestion()  
  
# 定义问题  
question = """  
现在笼子里有鸡（雉）和兔子在一起。从上面数一共有三十五个头，从下面数一共有九十四只脚，问一共有多少只鸡、多少只兔子？
使用中文回答 
"""  
  
# 解决问题  
response = math_question.forward(question=question)  
  
response.answer

'一共有23只鸡，12只兔子。'

In [4]:
turbo.inspect_history(n=3)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: 现在笼子里有鸡（雉）和兔子在一起。从上面数一共有三十五个头，从下面数一共有九十四只脚，问一共有多少只鸡、多少只兔子？ 使用中文回答
Reasoning: Let's think step by step in order to Question: 现在笼子里有鸡（雉）和兔子在一起。从上面数一共有三十五个头，从下面数一共有九十四只脚，问一共有多少只鸡、多少只兔子？ 使用中文回答
Reasoning: Let's think step by step in order to produce the answer. We need to set up a system of equations based on the given information. 

1. Let \( x \) be the number of chickens and \( y \) be the number of rabbits.
2. From the problem, we know:
   - The total number of heads is 35: \( x + y = 35 \)
   - The total number of feet is 94: \( 2x + 4y = 94 \)

3. We can simplify the second equation by dividing everything by 2:
   - \( x + 2y = 47 \)

4. Now we have the system of equations:
   - \( x + y = 35 \)
   - \( x + 2y = 47 \)

5. Subtract the first equation from th

"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: 现在笼子里有鸡（雉）和兔子在一起。从上面数一共有三十五个头，从下面数一共有九十四只脚，问一共有多少只鸡、多少只兔子？ 使用中文回答\nReasoning: Let's think step by step in order to\x1b Question: 现在笼子里有鸡（雉）和兔子在一起。从上面数一共有三十五个头，从下面数一共有九十四只脚，问一共有多少只鸡、多少只兔子？ 使用中文回答\nReasoning: Let's think step by step in order to produce the answer. We need to set up a system of equations based on the given information. \n\n1. Let \\( x \\) be the number of chickens and \\( y \\) be the number of rabbits.\n2. From the problem, we know:\n   - The total number of heads is 35: \\( x + y = 35 \\)\n   - The total number of feet is 94: \\( 2x + 4y = 94 \\)\n\n3. We can simplify the second equation by dividing everything by 2:\n   - \\( x + 2y = 47 \\)\n\n4. Now we have the system of equations:\n   - \\( x + y = 35 \\)\n   - \\( x + 2y 